# Serverless Task Migration
This app is designed to help identify tasks running in your snowflake that are underutilizing a warehouse so that they can be changed to a serverless task. This is designed to migrate these tasks to improve the proficiency of these services as well as save on the cost of execution, as an under-utilized warehouse often charges for downtime when spun up for short tasks

This Notedbook will drop a streamlit file at SIT_SOLUTIONS.STM.CODE_STAGE and install the app SIT_SOLUTIONS.STM.STM_STREAMLIT.

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import pickle
import codecs
import os
import base64

from snowflake.snowpark.context import get_active_session
session = get_active_session()
session.sql(f"""ALTER SESSION SET QUERY_TAG = '{{"origin":"sf_sit","name":"sit_serveless_task_migration","version":{{"major":1, "minor":0}} }}'""").collect()

current_database = session.sql("SELECT CURRENT_DATABASE()").collect()[0][0]
current_schema = session.sql("SELECT CURRENT_schema()").collect()[0][0]
try:
    # session.sql("SELECT * from INSTALL_TEST.TEST.TEST_OPERATIONS").collect()
    st.session_state.opps_test_info = session.sql(f"SELECT * from {current_database}.{current_schema}.TEST_OPERATIONS").to_pandas()
    run_mode = "test"
except:
    st.success("Running Customer Install. Please follow the steps!")
    run_mode = "customer"

Please choose the location you'd like to install to

In [ ]:
if run_mode == 'test':
    opps_info = st.session_state.opps_test_info
    install_db = opps_info[opps_info["KEY"] == "INSTALL_DB"]["VALUE"].values[0]
    admin_schema = opps_info[opps_info["KEY"] == "ADMIN_SCHEMA"]["VALUE"].values[0]
    results_schema = opps_info[opps_info["KEY"] == "RESULT_SCHEMA"]["VALUE"].values[0]
    ops_schema = opps_info[opps_info["KEY"] == "OPS_SCHEMA"]["VALUE"].values[0]
    install_stage = opps_info[opps_info["KEY"] == "INSTALL_STAGE"]["VALUE"].values[0]
    ops_wh = opps_info[opps_info["KEY"] == "OPS_WH"]["VALUE"].values[0]
elif run_mode =="customer":
    install_db = st.text_input("Database")
    admin_schema = st.text_input("Admin Schema")
    results_schema = st.text_input("Results Schema")
    ops_schema = st.text_input("Operations Schema")
    install_stage = st.text_input("Install Stage (for hosting code files)")
    ops_wh = st.text_input("User Warehouse")

In [ ]:
CREATE OR REPLACE PROCEDURE {{install_db}}.{{admin_schema}}.PUT_TO_STAGE(STAGE VARCHAR, FILENAME VARCHAR, CONTENT VARCHAR)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION=3.8
PACKAGES=('snowflake-snowpark-python')
HANDLER='put_to_stage'
COMMENT='{"origin": "sf_sit","name": "sit_serveless_task_migration","version": "{major: 1, minor: 0}"}'
AS $$
import io
import os

def put_to_stage(session, stage, filename, content):
    local_path = '/tmp'
    local_file = os.path.join(local_path, filename)
    f = open(local_file, "w", encoding='utf-8')
    f.write(content)
    f.close()
    session.file.put(local_file, '@'+stage, auto_compress=False, overwrite=True)
    return "saved file "+filename+" in stage "+stage
$$;


In [ ]:
"""gASVcRoAAAAAAAB9lIwFZmlsZXOUfZQojAguZ2l0a2VlcJRDAJSMD2Vudmlyb25tZW50LnltbJRD
Q25hbWU6IHNmX2VudgpjaGFubmVsczoKLSBzbm93Zmxha2UKZGVwZW5kZW5jaWVzOgotIHN0cmVh
bWxpdD0xLjM5LjCUjBxzZXJ2ZXJsZXNzX3Rhc2tfbWlncmF0aW9uLnB5lELnFgAAIyBJbXBvcnQg
cHl0aG9uIHBhY2thZ2VzCmltcG9ydCBzdHJlYW1saXQgYXMgc3QKaW1wb3J0IHBhbmRhcyBhcyBw
ZAppbXBvcnQgdGltZQpmcm9tIHNub3dmbGFrZS5zbm93cGFyay5jb250ZXh0IGltcG9ydCBnZXRf
YWN0aXZlX3Nlc3Npb24KCnN0LnNldF9wYWdlX2NvbmZpZyhsYXlvdXQ9IndpZGUiKQojIFdyaXRl
IGRpcmVjdGx5IHRvIHRoZSBhcHAKc3QuaW1hZ2UoJ2h0dHBzOi8vcmF3LmdpdGh1YnVzZXJjb250
ZW50LmNvbS9Tbm93Zmxha2UtTGFicy9lbWVyZ2luZy1zb2x1dGlvbnMtdG9vbGJveC9tYWluL2Jh
bm5lcl9lbWVyZ2luZ19zb2x1dGlvbnNfdG9vbGJveC5wbmcnKQoKc3QudGl0bGUoIlNlcnZlcmxl
c3MgVGFza3MgTWlncmF0aW9uIikKIyBHZXQgdGhlIGN1cnJlbnQgY3JlZGVudGlhbHMKc2Vzc2lv
biA9IGdldF9hY3RpdmVfc2Vzc2lvbigpCgpjc3MgPSAiIiIKPHN0eWxlPgogICAgLypVU0VTIFRI
RSBIQVMgU0VMRUNUT1IgVE8gRklMVEVSIEJZIFRIRSBORVNURUQgQ09OVEFJTkVSKi8KCmRpdlty
b2xlPSJkaWFsb2ciXTpoYXMoLnN0LWtleS1taWdyYXRlX2NvbnRhaW5lcil7CiAgICB3aWR0aDo4
NSU7Cn0KPC9zdHlsZT4iIiIKc3QuaHRtbChjc3MpCgoKQHN0LmRpYWxvZygiTWlncmF0ZSBUYXNr
IikKZGVmIG1pZ3JhdGVfdGFzayh0YXNrX25hbWUsIHdoX25hbWUsIHdoX3NpemUpOgogICAgd2l0
aCBzdC5jb250YWluZXIoa2V5PSJtaWdyYXRlX2NvbnRhaW5lciIpOgogICAgICAgIHN0LndyaXRl
KGYiVGhpcyBNaWdyYXRpb24gd2lsbCByZW1vdmUgdGFzayB7dGFza19uYW1lfSIpCiAgICAgICAg
c3Qud3JpdGUoZiJmcm9tIHdhcmVob3VzZSB7d2hfbmFtZX0sIGFuZCBtYWtlIGl0IHNlcnZlcmxl
c3Mgd2l0aCB0aGUgYmVsb3cgc2V0dGluZyIpCiAgICAgICAgd2hfc2l6ZV9saXN0ID0gWyJYU01B
TEwiLCJTTUFMTCIsIk1FRElVTSIsIkxBUkdFIiwiWExBUkdFIiwiWFhMQVJHRSIsIlhYWExBUkdF
Il0KICAgICAgICB3aF9zaXplID0gc3Quc2VsZWN0Ym94KCJJbml0aWFsIFdhcmVob3VzZSBTaXpl
Iix3aF9zaXplX2xpc3QsaW5kZXg9d2hfc2l6ZV9saXN0LmluZGV4KHdoX3NpemUucmVwbGFjZSgn
LScsJycpLnVwcGVyKCkpKQogICAgICAgIF9fLGNvbDIgPSBzdC5jb2x1bW5zKCgxMCwyKSkKICAg
ICAgICBpZiBjb2wyLmJ1dHRvbigiTWlncmF0ZSIsIHR5cGU9InByaW1hcnkiKToKICAgICAgICAg
ICAgc2Vzc2lvbi5zcWwoZiJBTFRFUiBUQVNLIHt0YXNrX25hbWV9IFNVU1BFTkQiKS5jb2xsZWN0
KCkKICAgICAgICAgICAgc2Vzc2lvbi5zcWwoZiJBTFRFUiBUQVNLIHt0YXNrX25hbWV9IFVOU0VU
IFdBUkVIT1VTRSIpLmNvbGxlY3QoKQogICAgICAgICAgICBzZXNzaW9uLnNxbChmIkFMVEVSIFRB
U0sge3Rhc2tfbmFtZX0gU0VUIFVTRVJfVEFTS19NQU5BR0VEX0lOSVRJQUxfV0FSRUhPVVNFX1NJ
WkUgPSAnWFNNQUxMJzsiKS5jb2xsZWN0KCkKICAgICAgICAgICAgc2Vzc2lvbi5zcWwoZiIiIkFM
VEVSIFRBU0sge3Rhc2tfbmFtZX0gU0VUIENPTU1FTlQgPSAne3sib3JpZ2luIjogInNmX3NpdCIs
Im5hbWUiOiAic2l0X3NlcnZlbGVzc190YXNrX21pZ3JhdGlvbiIsInZlcnNpb24iOiAie3ttYWpv
cjogMSwgbWlub3I6IDB9fSJ9fSciIiIpLmNvbGxlY3QoKQogICAgICAgICAgICBzZXNzaW9uLnNx
bChmIkFMVEVSIFRBU0sge3Rhc2tfbmFtZX0gUkVTVU1FIikuY29sbGVjdCgpCiAgICAgICAgICAg
IHN0LnN1Y2Nlc3MoZiJ7dGFza19uYW1lfSBNaWdyYXRlZCEiKQogICAgICAgICAgICB0aW1lLnNs
ZWVwKDUpCiAgICAgICAgICAgIHN0LnJlcnVuKCkKICAgICAgICAgICAgCgpAc3QuY2FjaGVfZGF0
YQpkZWYgZ2V0X3dhcmVob3VzZXMod2hfbmFtZXMpOgogICAgcmV0dXJuIHNlc3Npb24uc3FsKGYi
IiIKICAgIFNFTEVDVCBESVNUSU5DVCBXQVJFSE9VU0VfTkFNRSwgQ09OQ0FUKHQuREFUQUJBU0Vf
TkFNRSwnLicsdC5TQ0hFTUFfTkFNRSwnLicsIHQuTkFNRSkgYXMgRlVMTF9OQU1FLCB0Lm5hbWUg
RlJPTSBTTk9XRkxBS0UuQUNDT1VOVF9VU0FHRS5UQVNLX0hJU1RPUlkgdAogIEpPSU4gU05PV0ZM
QUtFLkFDQ09VTlRfVVNBR0UuUVVFUllfSElTVE9SWSBxIE9OIHQuUVVFUllfSUQgPSBxLlFVRVJZ
X0lECiAgUklHSFQgSk9JTiBTTk9XRkxBS0UuQUNDT1VOVF9VU0FHRS5NRVRFUklOR19ISVNUT1JZ
IHcgT04gcS53YXJlaG91c2VfbmFtZSA9IHcubmFtZQogIFdIRVJFIAogIFFVRVJZX1NUQVJUX1RJ
TUUgQkVUV0VFTiB3LlNUQVJUX1RJTUUgQU5EIHcuRU5EX1RJTUUKICBBTkQgTk9UIENPTlRBSU5T
KFdBUkVIT1VTRV9OQU1FLCdDT01QVVRFX1NFUlZJQ0UnKSAKICBBTkQgU0NIRURVTEVEX1RJTUUg
QkVUV0VFTiBkYXRlYWRkKERBWSwgLSAxNCwgQ1VSUkVOVF9USU1FU1RBTVAoKSkgQU5EIENVUlJF
TlRfVElNRVNUQU1QKCkKICBBTkQgdy5uYW1lIGluICh7d2hfbmFtZXN9KTsKICAgICIiIikudG9f
cGFuZGFzKCkKCmlmICJxdWFsX3Rhc2tzIiBub3QgaW4gc3Quc2Vzc2lvbl9zdGF0ZToKICAgIHdp
dGggc3Quc3Bpbm5lcigiRmV0Y2hpbmcgVGFza3MiKToKCiAgICAgICAgc3Quc2Vzc2lvbl9zdGF0
ZS5xdWFsX3Rhc2tzID0gc2Vzc2lvbi5zcWwoIiIiU0VMRUNUIERJU1RJTkNUIENPTkNBVCh0LkRB
VEFCQVNFX05BTUUsJy4nLHQuU0NIRU1BX05BTUUsJy4nLCBOQU1FKSBhcyBGVUxMX05BTUUsTkFN
RSwgdC5RVUVSWV9URVhULCBXQVJFSE9VU0VfTkFNRSwgV0FSRUhPVVNFX1NJWkUsIGlmZihEQVRF
RElGRigiTUlOVVRFIixRVUVSWV9TVEFSVF9USU1FLCBDT01QTEVURURfVElNRSkgPCAxLCAnVFJV
RScsJ0ZBTFNFJykgYXMgU0hPUlRfUlVOVElNRSwgaWZmKERBVEVESUZGKCJNSU5VVEUiLFFVRVJZ
X1NUQVJUX1RJTUUsIFNDSEVEVUxFRF9USU1FKSA+IDEsICdUUlVFJywnRkFMU0UnKSBhcyBSQU5f
T1ZFUiBGUk9NIFNOT1dGTEFLRS5BQ0NPVU5UX1VTQUdFLlRBU0tfSElTVE9SWSB0CiAgICAgICAg
ICBKT0lOIFNOT1dGTEFLRS5BQ0NPVU5UX1VTQUdFLlFVRVJZX0hJU1RPUlkgcSBPTiB0LlFVRVJZ
X0lEID0gcS5RVUVSWV9JRAogICAgICAgICAgV0hFUkUgCiAgICAgICAgICAoREFURURJRkYoIk1J
TlVURSIsUVVFUllfU1RBUlRfVElNRSwgQ09NUExFVEVEX1RJTUUpIDwgMQogICAgICAgICAgT1IK
ICAgICAgICAgIERBVEVESUZGKCJNSU5VVEUiLFFVRVJZX1NUQVJUX1RJTUUsIFNDSEVEVUxFRF9U
SU1FKSA+IDEpCiAgICAgICAgICBBTkQgTk9UIENPTlRBSU5TKFdBUkVIT1VTRV9OQU1FLCdDT01Q
VVRFX1NFUlZJQ0UnKSAKICAgICAgICAgIEFORCBTQ0hFRFVMRURfVElNRSBCRVRXRUVOIGRhdGVh
ZGQoREFZLCAtIDE0LCBDVVJSRU5UX1RJTUVTVEFNUCgpKSBBTkQgQ1VSUkVOVF9USU1FU1RBTVAo
KTsiIiIpLnRvX3BhbmRhcygpCgpxdWFsX3Rhc2tzID0gc3Quc2Vzc2lvbl9zdGF0ZS5xdWFsX3Rh
c2tzCgp3aXRoIHN0LmNvbnRhaW5lcihib3JkZXI9VHJ1ZSk6CiAgICBzdC53cml0ZSgiIiJUaGUg
dGFza3MgYmVsb3cgYXJlIHRoZSBmaXJzdCBnbGFuY2Ugb2YgdGFza3MgdGhhdCBtYXkgcXVhbGlm
eSBmb3IgbWlncmF0aW9uIHRvIHNlcnZlcmxlc3MgdGFza3MgXG4KICAgIFRoZXkgZWl0aGVyIHJ1
biBmb3IgdW5kZXIgYSBtaW51dGUsIG9yIHJ1biBsb25nZXIgdGhhbiB0aGUgdGltZSBiZXR3ZWVu
IHRoZWlyIHNjaGVkdWxlZCBydW4gYW5kIHRoZSBuZXh0IHJ1bgogICAgIiIiKQogICAgc3QuZGF0
YWZyYW1lKHF1YWxfdGFza3MsIHVzZV9jb250YWluZXJfd2lkdGg9VHJ1ZSwgY29sdW1uX2NvbmZp
Zz17J0ZVTExfTkFNRSc6Tm9uZX0pCgp3aXRoIHN0LmNvbnRhaW5lcihib3JkZXI9VHJ1ZSk6CiAg
ICBzdC53cml0ZSgiIiJUaGUgVGFza3MgdGhhdCBydW4gbG9uZyBpbiB0aGUgcHJldmlvdXMgbGlz
dCBhcmUgZ3JlYXQgY2FuZGlkYXRlcyBmb3Igc2VydmVybGVzcyB3YXJlaG91c2VzIGFzIHRoZXkg
Y2FuIGhhdmUgYSB0YXJnZXQgZmluaXNoIHNldCBhbmQgdGhlIHdhcmVob3VzZSB3aWxsIHNjYWxl
IGZvciB0aGUgc2NoZWR1bGUiIiIpCiAgICBzdC53cml0ZSgiIiJGb3Igc2hvcnQgcnVubmluZyB0
YXNrcywgdG8gZnVydGhlciBxdWFsaWZ5IHRoZW0gd2UnbGwgY2hlY2sgaG93IG1hbnkgdGFza3Mg
Z2VuZXJhbGx5IHJ1biBvbiB0aGUgc2FtZSB3YXJlaG91c2UiIiIpCgp3aXRoIHN0LmNvbnRhaW5l
cihib3JkZXI9VHJ1ZSk6CiAgICBzdC53cml0ZSgiIiJUaGUgYmVsb3cgbGlzdCBzaG93cyB0aGUg
bnVtYmVyIG9mIHRhc2tzIHRoYXQgYWxsIHJ1biBhcm91bmQgdGhlIHNhbWUgdGltZSBvbiBlYWNo
IHdhcmVob3VzZSIiIikKCiAgICB3YXJlaG91c2VzID0gc2V0KHF1YWxfdGFza3NbIldBUkVIT1VT
RV9OQU1FIl0udG9saXN0KCkpCiAgICB3YXJlaG91c2VzX2wgPSAiJ3t9JyIuZm9ybWF0KCInLCAn
Ii5qb2luKHdhcmVob3VzZXMpKQoKICAgIHdhcmVob3VzZV9saXN0ID0gZ2V0X3dhcmVob3VzZXMo
d2FyZWhvdXNlc19sKQogICAgc3QuZGF0YWZyYW1lKHdhcmVob3VzZV9saXN0LHVzZV9jb250YWlu
ZXJfd2lkdGg9VHJ1ZSkKCiAgICBzdC53cml0ZSgiRm9yIGFueSB0YXNrcyB0aGF0IHJ1biBmb3Ig
bGVzcyB0aGFuIGEgbWludXRlIG9uIGEgd2FyZWhvdXNlIGFsb25lIHNob3VsZCBiZSBtaWdyYXRl
ZCB0byBhIHNlcnZlcmxlc3Mgd2FyZWhvdXNlIikKICAgIHN0LndyaXRlKCJBZGRpdGlvbmFsbHkg
YW55IHRhc2tzIHNjaGVkdWxlZCB3aXRoIGEgdG90YWwgcnVudGltZSBvZiBsZXNzIHRoYW4gYSBm
ZXcgbWludXRlcyB3b3VsZCBsaWtlbHkgZWFjaCBiZW5lZml0IGZyb20gYmVpbmcgc2VydmVybGVz
cyIpCiAgICBzdC53cml0ZSgiUmV2aWV3IGVhY2ggd2FyZWhvdXNlIGJlbG93IGFuZCBtaWdyYXRl
IGFueSB0YXNrcyB0aGF0IGFyZSByZWNvbW1lbmRlZCBvciB0aGF0IHlvdSB3b3VsZCBsaWtlIHRv
IikKICAgIAogICAgZm9yIHdoX25hbWUgaW4gd2FyZWhvdXNlczoKICAgICAgICB3aF90YXNrcyA9
IHdhcmVob3VzZV9saXN0W3dhcmVob3VzZV9saXN0WyJXQVJFSE9VU0VfTkFNRSJdID09IHdoX25h
bWVdWyJOQU1FIl0udG9saXN0KCkKICAgICAgICB3aF9zaXplID0gcXVhbF90YXNrc1txdWFsX3Rh
c2tzWyJXQVJFSE9VU0VfTkFNRSJdID09IHdoX25hbWVdWyJXQVJFSE9VU0VfU0laRSJdLnZhbHVl
c1swXQogICAgICAgIHdpdGggc3QuZXhwYW5kZXIod2hfbmFtZSk6CiAgICAgICAgICAgIHN0Lndy
aXRlKCIjIyMiKQogICAgICAgICAgICBjb2wxLF9fLGNvbDIgPSBzdC5jb2x1bW5zKDMpCiAgICAg
ICAgICAgIGNvbDEuc3ViaGVhZGVyKCIqVGFzayBOYW1lKiIpCgogICAgICAgICAgICBmb3Igd2hf
dGFzayBpbiB3aF90YXNrczoKICAgICAgICAgICAgICAgIGZ1bGxfdGFza19uYW1lID0gd2FyZWhv
dXNlX2xpc3Rbd2FyZWhvdXNlX2xpc3RbIk5BTUUiXSA9PSB3aF90YXNrXVsiRlVMTF9OQU1FIl0u
dmFsdWVzWzBdCiAgICAgICAgICAgICAgICBkZXNjcmliZV90YXNrID0gcGQuRGF0YUZyYW1lKHNl
c3Npb24uc3FsKGYiREVTQ1JJQkUgVEFTSyB7ZnVsbF90YXNrX25hbWV9IikuY29sbGVjdCgpKQog
ICAgICAgICAgICAgICAgaWYgZGVzY3JpYmVfdGFza1sid2FyZWhvdXNlIl0udmFsdWVzWzBdOgog
ICAgICAgICAgICAgICAgICAgIHN0LmRpdmlkZXIoKQogICAgICAgICAgICAgICAgICAgIGNvbDEs
X18sY29sMiA9IHN0LmNvbHVtbnMoMykKICAgICAgICAgICAgICAgICAgICBjb2wxLndyaXRlKHdo
X3Rhc2spCiAgICAgICAgICAgICAgICAgICAgY29sMi5idXR0b24oIk1pZ3JhdGUiLG9uX2NsaWNr
PW1pZ3JhdGVfdGFzaywgYXJncz0oZnVsbF90YXNrX25hbWUsd2hfbmFtZSx3aF9zaXplKSwga2V5
PXdoX3Rhc2srIm1pZ3JhdGVfYnV0dG9uIikKICAgICAgICAKICAgIJSMFGluc3RhbGxfc29sdXRp
b24uc3FslELSAgAALS0hamluamEyCgpDUkVBVEUgT1IgUkVQTEFDRSBUQUJMRSB7eyBpbnN0YWxs
X2RiIH19Lnt7IG9wc19zY2hlbWEgfX0uVEVTVF9PUEVSQVRJT05TKEtFWSBTVFJJTkcsIFZBTFVF
IFNUUklORyk7CgpJTlNFUlQgSU5UTyB7eyBpbnN0YWxsX2RiIH19Lnt7IG9wc19zY2hlbWEgfX0u
VEVTVF9PUEVSQVRJT05TClZBTFVFUwooJ0lOU1RBTExfREInLCd7eyBpbnN0YWxsX2RiIH19Jyks
CignQURNSU5fU0NIRU1BJywne3sgYWRtaW5fc2NoZW1hIH19JyksCignT1BTX1NDSEVNQScsJ3t7
IG9wc19zY2hlbWEgfX0nKSwKKCdSRVNVTFRfU0NIRU1BJywne3sgcmVzdWx0c19zY2hlbWEgfX0n
KSwKKCdJTlNUQUxMX1NUQUdFJywne3sgaW5zdGFsbF9zdGFnZSB9fScpLAooJ09QU19XSCcsJ3t7
IG9wc193aCB9fScpOwoKQ1JFQVRFIE9SIFJFUExBQ0UgU1RSRUFNTElUIHt7IGluc3RhbGxfZGIg
fX0ue3sgb3BzX3NjaGVtYSB9fS5TVE1fU1RSRUFNTElUClJPT1RfTE9DQVRJT04gPSAnQHt7aW5z
dGFsbF9kYn19Lnt7b3BzX3NjaGVtYX19Lnt7aW5zdGFsbF9zdGFnZX19JwpNQUlOX0ZJTEUgPSAn
c2VydmVybGVzc190YXNrX21pZ3JhdGlvbi5weScKUVVFUllfV0FSRUhPVVNFID0ge3tvcHNfd2h9
fQpDT01NRU5UPSd7Im9yaWdpbiI6ICJzZl9zaXQiLCJuYW1lIjogInNpdF9zZXJ2ZWxlc3NfdGFz
a19taWdyYXRpb24iLCJ2ZXJzaW9uIjogInttYWpvcjogMSwgbWlub3I6IDB9In0nOwqUdXMu
"""

In [ ]:
import pickle
import codecs
import os
import base64

from snowflake.snowpark.context import get_active_session
session = get_active_session()

def decode(obj):
    pickled = codecs.decode(obj.encode(), 'base64')
    return pickle.loads(pickled)


# Decode object to confirm it is the same as the original object
decoded_obj = decode(encoded_obj)

# Stage each file
for file_name, file_content in decoded_obj['files'].items():
    # replace double dollars to help with SQL compatibility
    file_content = str(file_content.decode("utf-8")).replace("$$", '"""')
    sql = """call {db}.{schema}.put_to_stage('{db}.{schema}.{stage}','{file_name}', $${file_content}$$)""".format(
        file_name=file_name, 
        file_content=file_content,
        db=install_db,
        schema=admin_schema,
        stage=install_stage)
    session.sql(sql).collect()

# Stage each image

In [ ]:
CREATE OR REPLACE STREAMLIT {{install_db}}.{{ops_schema}}.STM_STREAMLIT
ROOT_LOCATION = '@{{install_db}}.{{ops_schema}}.{{install_stage}}'
MAIN_FILE = 'serverless_task_migration.py'
QUERY_WAREHOUSE = {{ops_wh}}
COMMENT='{"origin": "sf_sit","name": "sit_serveless_task_migration","version": "{major: 1, minor: 0}"}';